In [ ]:
!pip install transformers==4.4.0
!pip install sentencepiece==0.1.94
!pip install datasets
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
!pip install tokenizer==0.9.4

In [ ]:
!nvidia-smi

Wed Apr 28 14:41:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
from transformers import (
    AdamW,
    MT5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup

)

In [ ]:
pl.seed_everything(42)

Global seed set to 42


42

<h1> Download Thaisum Dataset </h1>

In [ ]:
dataset = load_dataset('thaisum')

Dataset thaisum downloaded and prepared to /root/.cache/huggingface/datasets/thaisum/thaisum/1.0.0/347b33c852af4d796e1224e00e15142d626608a9fa3e07ad6d19dfd8fcae5423. Subsequent calls will reuse this data.


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 358868
    })
    validation: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 11000
    })
    test: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 11000
    })
})

In [ ]:
dataset['train'][2]

{'body': ' โดยล่าสุดถูกปั่นว่าเรียกร้องให้สังคมให้อภัยเปรี้ยว6 มิ.ย. 2560 ผู้สื่อข่าวรายงานว่า เมื่อเวลา 21.59 น. ที่ผ่านมาอังคณา นีละไพจิตรกรรมการสิทธิมนุษยชนแห่งชาติ โพสต์ภาพ พร้อมข้อความผ่านเฟซบุ๊ก ซึ่งเป็นภาพบิดเบือนข้อมูลเกี่ยวกับ อังคณา พร้อระบุว่า ทำงานตรวจสอบการละเมิดสิทธิฯมานาน พรุ่งนี้(7 มิ.ย.60) 10.00 น. จะไปแจ้ง กองบังคับการปราบปรามการกระทำความผิดเกี่ยวกับอาชญากรรมทางเทคโนโลยี (ปอท) ศูนย์ราชการเฉลิมพระเกียรติ ชั้น 4 อาคาร B เรื่องการถูกละเมิดสิทธิ และศักดิ์ศรีความเป็นมนุษย์ ทำให้ถูกเกลียดชังโดยการใช้ภาพ และข้อความเผยแพร่ทาง facebook ต่างๆ โดยล่าสุด คือกรณี เปรี้ยว ทั้งที่ไม่ได้อยู่ประเทศไทยในช่วงเวลาดังกล่าว ทั้งนี้คำพูดเพื่อสร้างความเกลียดชัง (HateSpeech) บนแอพพลิเคชั่น เริ่มมีมากขึ้นตั้งแต่ทำหน้าที่ กรรมการสิทธิฯ ในฐานะที่รับผิดชอบ สิทธิพลเมืองและสิทธิทางการเมือง และสิทธิมนุษยชนในจังหวัดชายแดนภาคใต้ ที่มีหน้าที่ต้องตรวจสอบข้อเท็จจริงกรณีเรื่องร้องเรียนเรื่องการละเมิดสิทธิฯต่างๆ โดยเฉพาะอย่างยิ่ง การละเมิดที่เกิดจากการกระทำของเจ้าหน้าที่รัฐบางคน',
 'summary': 'อังคณา กสม. 

<h1> Convert Dataset to Pandas DataFrame. </h1>

In [ ]:
def get_df(dataset, set_type):
  data_list = []
  for data in dataset[set_type]:
    data_list.append({'body':data['body'], 'summary':data['summary'], 'tags':data['tags'].replace(',','').strip(), 'title':data['title'], 'type':data['type']})
  return pd.DataFrame(data_list)

In [ ]:
train_df = get_df(dataset,'train')
val_df = get_df(dataset,'validation')
test_df = get_df(dataset,'test')

In [ ]:
train_df.head()

,body,summary,tags,title,type
0,เมื่อวันที่ 6 ม.ค.60 ที่ทำเนียบรัฐบาล นายวิษณุ...,วิษณุ ยันโรดแม็ปตามขั้นตอนเดิม เชื่อ สนช.หยุดพ...,เลือกตั้งโรดแม็ปวิษณุ เครืองามร่างรัฐธรรมนูญสนช.,วิษณุ ยันโรดแม็ปเดิม ตอบไม่ถูกเวลาเลือกตั้ง ต...,
1,พ่ายสวีเดนด้วยประตูรวม 0–1 ตกรอบเพลย์ออฟไปอย่...,รอกันมา 2 สัปดาห์ บอลสโมสรลีกต่างๆของยุโรปก็กล...,ตะลุยฟุตบอลโลกโต้ บ้านแหลมบราซิลเยอรมนีสถิติ,แชมป์โลก 2018,
2,โดยล่าสุดถูกปั่นว่าเรียกร้องให้สังคมให้อภัยเป...,อังคณา กสม. โพสต์ระบุเตรียมเข้าร้องต่อ ปอท.เรื...,ปอท.สร้างความเกลียดชังอังคณา นีละไพจิตรเปรี้ยว,อังคณา เผยเตรียมร้อง ปอท. หลังถูกโพสต์สร้างควา...,"สิทธิมนุษยชน,ไอซีที"
3,วันนี้ (7 เม.ย.2559) นายโชคดี อมรวัฒน์ รองผู้ว...,รองผู้ว่าราชการจังหวัดภูเก็ตลงพื้นที่ตรวจสอบเก...,บุกรุกภูเก็ตเกาะนาคาน้อยหิรัญพฤกษ์ThaiPBSnewsข...,รองผู้ว่าฯภูเก็ตตรวจสอบที่ดินบนเกาะนาคาน้อย กร...,ภูมิภาค
4,สำนักข่าวต่างประเทศรายงานวันที่ 26 ก.ย. ว่า หล...,อันโตนิโอ คอนเต สร้างปรากฏการณ์ให้กับทัพ งูใหญ...,อินเตอร์ มิลานอันโตนิโอ คอนเตกัลโช เซเรีย อางู...,คอนเต คนแรกทำสถิติให้ งูใหญ่ สตาร์ทเข้าวินรวด ...,"กีฬา,ฟุตบอลยุโรป"


In [ ]:
train_df = train_df[:50000]
val_df = val_df[:5000]
test_df = test_df[:5000]

In [ ]:
train_df = train_df.drop_duplicates(subset=["body"]).reset_index(drop=True)

<h1> Download Pre-trained Model.

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("Pollawat/mt5-small-thai-qg")

<h1> Text to Sequence. </h1>

In [ ]:
class ThaiSumDataset(Dataset):

  def __init__(
      self,
      data: pd.DataFrame,
      tokenizer: T5Tokenizer,
      source_max_token_len: int = 50,
      target_max_token_len: int = 150
  ):

      self.tokenizer = tokenizer
      self.data = data
      self.source_max_token_len = source_max_token_len
      self.target_max_token_len = target_max_token_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    source_encoding = tokenizer(
        data_row["tags"],
        max_length = self.source_max_token_len,
        padding = 'max_length',
        truncation = True,
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors='pt'
    )

    target_encoding = tokenizer(
        data_row["summary"],
        max_length = self.target_max_token_len,
        padding = 'max_length',
        truncation = True,
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors='pt'
    )

    labels = target_encoding["input_ids"]
    labels[labels == 0] = -100

    return dict(
        tags = data_row["tags"],
        body = data_row["summary"],
        input_ids = source_encoding["input_ids"].flatten(),
        attention_mask = source_encoding["attention_mask"].flatten(),
        labels=labels.flatten()

    )

In [ ]:
sample_dataset = ThaiSumDataset(train_df, tokenizer)

In [ ]:
for data in sample_dataset:
  print(data['tags'])
  print(data['body'])
  print(data['input_ids'][:10])
  print(data['labels'][:10])
  break

เลือกตั้งโรดแม็ปวิษณุ เครืองามร่างรัฐธรรมนูญสนช.
วิษณุ ยันโรดแม็ปตามขั้นตอนเดิม เชื่อ สนช.หยุดพูดขยับเลือกตั้ง ปัดวิจารณ์ ยึดตามกรอบเวลา ย้ำเริ่มนับโรดแม็ปเมื่อ รธน.ประกาศใช้
tensor([   259, 238131,  42859,   6088, 237720,   9964,  32816,  97652,  20390,
         60612])
tensor([   259,  32816,  97652,  20390,  60612,    259, 190560,  42859,   6088,
        237720])


In [ ]:
sample_dataset[10]['labels'].shape

torch.Size([150])

<h1> DataFrame to DataLoader. </h1>

In [ ]:
class ThaiSumModule(pl.LightningDataModule):

  def __init__(
      self,
      train_df: pd.DataFrame,
      val_df: pd.DataFrame,
      test_df: pd.DataFrame,
      tokenizer: T5Tokenizer,
      batch_size: int = 8,
      source_max_token_len: int = 50,
      target_max_token_len:int = 150
  ):
      super().__init__()
      self.batch_size = batch_size
      self.train_df = train_df
      self.val_df = val_df
      self.test_df = test_df
      self.tokenizer = tokenizer
      self.source_max_token_len = source_max_token_len
      self.target_max_token_len = target_max_token_len

  def setup(self):
    self.train_dataset = ThaiSumDataset(
        self.train_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
    )

    self.val_dataset = ThaiSumDataset(
        self.val_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
    )

    self.test_dataset = ThaiSumDataset(
        self.test_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
    )

  def train_dataloader(self):
    return DataLoader(
          self.train_dataset,
          batch_size = self.batch_size,
          shuffle = True,
          num_workers = 4
      )

  def val_dataloader(self):
    return DataLoader(
          self.val_dataset,
          batch_size = 1,
          num_workers = 4
      )

  def test_dataloader(self):
    return DataLoader(
          self.test_dataset,
          batch_size = 1,
          num_workers = 4
      )

<h1> Tuning parameters.

In [ ]:
BATCH_SIZE = 8
N_EPOCHS = 5
SOURCE_MAX_TOKEN_LEN = 50
TARGET_MAX_TOKEN_LEN = 150

data_module = ThaiSumModule(train_df, val_df, test_df, tokenizer, batch_size=BATCH_SIZE, source_max_token_len=SOURCE_MAX_TOKEN_LEN, target_max_token_len=TARGET_MAX_TOKEN_LEN)
data_module.setup()

In [ ]:
MODEL_NAME = "Pollawat/mt5-small-thai-qg"

model = MT5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)

##Modeling

In [ ]:
class FakeNewsModel(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.model = MT5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels
    )

    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log('train_loss', loss, prog_bar=True, logger=True)
    return loss

  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log('val_loss', loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log('test_loss', loss, prog_bar=True, logger=True)
    return loss

  def configure_optimizers(self):
    return AdamW(self.parameters(), lr=0.0001)

In [ ]:
model = FakeNewsModel()

<h1> Train Model. </h1>

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

In [ ]:
logger = TensorBoardLogger('training-logs', name='fake-news-gen')
trainer = pl.Trainer(
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate=30
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
# %load_ext tensorboard

In [ ]:
# %tensorboard --logdir ./lightning_logs

In [ ]:
# !rm -rf lightning_logs

In [ ]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                        | Params
------------------------------------------------------
0 | model | MT5ForConditionalGeneration | 300 M 
------------------------------------------------------
300 M     Trainable params
0         Non-trainable params
300 M     Total params
1,200.707 Total estimated model params size (MB)


Global seed set to 42


1

In [ ]:
# trainer.test()

##Prediction

save model to google drive.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# model.model.save_pretrained('/content/drive/MyDrive/master_com_science/Semester_2_2020/NLP/project/saved_model/')
# model.model.save_pretrained('./saved_model/')

load model from google drive.

In [ ]:
# MODEL_NAME = './saved_model/'
# trained_model = FakeNewsModel()

In [ ]:
trained_model = model

In [ ]:
trained_model = FakeNewsModel.load_from_checkpoint("checkpoints/best-checkpoint.ckpt")
trained_model.freeze()

In [ ]:
def generate_answer(tags):
    source_encoding = tokenizer(
        tags,
        max_length = 150,
        padding = 'max_length',
        truncation = True,
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors='pt'
    )
    print(source_encoding)
    generated_ids = trained_model.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_beams=5,
        max_length=150,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True,
        use_cache=True
    )

    preds = [
             tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
             for generated_id in generated_ids
    ]

    return "".join(preds)


In [ ]:
generate_answer("ฟุตบอล")

{'input_ids': tensor([[29898,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

'ศึกฟุตบอลชิงถ้วยพระราชทานสมเด็จพระเจ้าอยู่หัวฯ ที่สนามกีฬามวยไทยลีก 1 (เมฆ) มีมติเห็นชอบให้ยกเลิกการคัดเลือกการแข่งขันการแข่งขันฟุตบอลโลกครั้งที่ 2'